## Pytorch Loss Function 

相关链接：
- Pytorch的损失函数：https://blog.csdn.net/rosefun96/article/details/88058708
- Pytorch1.8官方英文Loss Function文档：https://pytorch.org/docs/stable/nn.html#loss-functions

### 一、损失函数简介
- 损失函数，即目标函数，用于计算真实值和预测值之间差异的函数，和优化器是编译一个神经网络模型的重要要素。
- 损失函数返回的值是标量（向量无法直接比较，向量可以通过范式等标量来比较）
- 常见的损失函数，如下
    - 绝对值损失
    - 平方损失
    - 对数损失

### 二、损失函数的本质
    任何一个有负对数似然组成的损失都是定义在训练集上的经验分布和定义在模型上的概率分布的交叉熵。

### 三、损失函数实例
    定义预测值`sample`和目标值`target`，使用不同的损失函数计算其损失。


In [14]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

sample = Variable(torch.ones(2, 2))
target = Variable(torch.tensor([[0, 1], [2, 3]]))

sample, target

(tensor([[1., 1.],
         [1., 1.]]),
 tensor([[0, 1],
         [2, 3]]))

### 1. 绝对值损失
----
#### 1.1 [nn.L1Loss](https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html?highlight=l1loss#torch.nn.L1Loss)
    L1绝对值损失使用预测值与真实值的绝对误差的平均数。常用于回归问题。如下公式，其中`N`为Batch Size：

$$l(x,y)=L={ \left[ l_1, l_2, ..., l_N \right] }^T, l_n = |x_n - y_n|$$

$l(x,y) = sum(L)$, `reduction='sum'`

$$L(x, y)=\sum_{n=1}^{N} |x_n - y_n|$$

In [8]:
criterion_sum = nn.L1Loss(reduction='sum')
loss_sum = criterion_sum(sample, target)
loss_sum

tensor(4.)

$l(x,y)=mean(L)$ , `reduction='mean' or None` _The sum operation still operates over all the elements, and divides by N ._

$$L(x, y)=\sum_{n=1}^{N} \frac1N |x_n - y_n|$$

In [18]:
criterion_mean = nn.L1Loss(reduction='mean')
loss_mean = criterion_mean(sample, target)
loss_mean

tensor(1.)

##### 代码实例：全1矩阵与全0矩阵计算绝对值损失，然后计算梯度并反向传播

In [20]:
zeros = torch.zeros(3, 5, requires_grad=True)
ones = torch.ones(3, 5)

output_mean = criterion_mean(zeros, ones)
output_sum = criterion_sum(zeros, ones)

print("gradient before backward: ", zeros.grad)

output_mean.backward()
output_sum.backward()

print("gardient after backward: ", zeros.grad)

output_mean, output_sum

gradient before backward:  None
gardient after backward:  None


(tensor(1., grad_fn=<L1LossBackward>), tensor(15., grad_fn=<L1LossBackward>))

#### 1.2  [nn.SmoothL1Loss](https://pytorch.org/docs/stable/generated/torch.nn.SmoothL1Loss.html?highlight=smoothl1loss#torch.nn.SmoothL1Loss)
平滑 L1 损失（Smooth L1 Loss）。创建一个标准，如果绝对逐元素误差（absolute element-wise error）下降到`beta`以下，则使用平方损失。否则，使用 L1 损失。

- 它对离群点（outliers）的敏感性不如（less sensitive）`torch.nn.MSELoss`平方误差损失，并且在某些情况下（in some cases）可以防止梯度爆炸（exploding gradients）请参阅(e.g. see Fast R-CNN paper by Ross Girshick)。
- 忽略比例因子（Omitting a scaling facotr of beta），即`beat = 1`，又称 Huber Loss。
- 当比例因子`beta = 0` 时，等价于L1损失。
    
$$loss(x, y) = \frac1n \sum_i{z_i}$$

$$ z_i =
    \begin{cases}
        0.5 (x_i - y_i)^2/beta, & \text{if $|x_i - y_i| < beta$} \\
        |x_i - y_i| - 0.5 * beta, & \text{otherwise} \\
    \end{cases}
$$

###### 讨论：为什么使用 Huber Loss?
    L2平方损失的结果是算数均值无偏估计，L1绝对值损失的结果是中值无偏估计。但是L2损失容易受异常点影响，HUber Loss 在0点附近是强凸的，结合了平方损失和绝对值损失的优点。

In [15]:
criterion = nn.SmoothL1Loss(reduction='mean')
loss = criterion(sample, target)
print(loss)

tensor(0.6250)


### 2.平方损失
----
#### 2.1 [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html?highlight=mseloss#torch.nn.MSELoss)
    L2损失，又称平方损失（Mean Squared Error），用于计算预测值与真实值之间的平方和的平均数。常用于回归问题。
    
$$l(x,y)=L={ \left[ l_1, l_2, ..., l_N \right] }^T, l_n = (x_n - y_n)^2$$

$$
l(x,y) =
    \begin{cases}
        mean(L), & \text{if reduction = 'mean'} \\
        sum(L), & \text{if reduction = 'sum'} \\
    \end{cases}
$$

In [17]:
criterion = nn.MSELoss(reduction='mean')
loss = criterion(sample, target)
loss

tensor(1.5000)

### 3. 对数损失

- 信息量：将一份可能发生的信息，用1/e来衡量。（可能有偏差，但不影响整体理解）
- 信息熵：所有可能发生事件的信息量的总和，再取`-logP(x)`，信息发生的概率越大，信息量越小。
- 交叉熵：衡量实际输出的概率与目标输出概率分布的距离。
- 相对熵（KL散度）：衡量两个概率分布之间的差异。

----

#### 3.1 [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html?highlight=nllloss#torch.nn.NLLLoss)
负对数似然损失（Negative Log Likeihood），也用于分类。
- 通过在网络的最后一层添加LogSoftmax层，可以轻松地获得神经网络中的对数概率。如果您不希望添加额外的图层，则可以改用CrossEntropyLoss。

$$l(x,y)=L={ \left[ l_1, l_2, ..., l_N \right] }^T$$

$$l_n = -w_{y_n} x_{n,y_n}$$

$$w_c = \text{weight}[c]·1 \left\{ c≠\text{ignore_index} \right\}$$

$$
l(x,y) =
    \begin{cases}
        \sum_{n=1}^N \frac {1}{\sum_{n=1}^N w_{yn}} l_n), & \text{if reduction = 'mean'} \\
        \sum_{n=1}^N l_n, & \text{if reduction = 'sum'} \\
    \end{cases}
$$

- nn.NLLLoss2d 目前在Pytorch 1.8中没有说明文档。在用全卷积做分类时，最后图片的每个点都会预测一个类别标签。input (N, C, H, W)，target (N, H, W)

NLL Loss + LogSoftmax

In [34]:
torch.manual_seed(0)
sample = torch.randn(3, 5)
target = torch.tensor([1, 0, 4])

m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
output = loss(m(sample), target)
sample, output

(tensor([[ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845],
         [-1.3986,  0.4033,  0.8380, -0.7193, -0.4033],
         [-0.5966,  0.1820, -0.8567,  1.1006, -1.0712]]),
 tensor(2.7184))

CE Loss

In [36]:
loss = nn.CrossEntropyLoss()
output = loss(sample, target)
output

tensor(2.7184)

手动计算 NLL Loss + LogSoftmax

In [46]:
def LogSoftmax(x):
    return torch.log(torch.exp(x) / torch.sum(torch.exp(x),axis=1,keepdims=True))

log_softmax_input = LogSoftmax(sample)
loss_sum = 0.
c = sample.shape[0]
for i in range(c):
    loss_sum += - log_softmax_input[i][target[i]]
loss_mean = loss_sum / c
loss_mean

tensor(2.7184)

#### 3.2 [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)
   交叉熵损失，用于计算实际输出概率分布与目标输出概率分布的距离，交叉熵越小，两个概率分布就越接近。交叉熵损失组合了 [LogSoftmax](https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html#torch.nn.LogSoftmax) 和 [NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) 。

- `weight`：提供一维可选参数（大小为`C`）对应每个类别的权重，当使用非平衡训练集时，交叉熵损失非常有用。默认`weight`都为1。
- `input`：预期输入包含每个类的原始、未标准化的分数。`input`大小必须为`(minibatch, C)`或者`(minibatch, C, d1, d2, ..., dK), K≥1`。`input`可以是更高维度的输入，比如二维图像，`K`是图像尺寸。在这种情况下，它计算每个像素的 NLL Loss。
- `ignore_index`：忽略索引，该索引的目标值将被忽略且不会影响输入梯度。对于每个`minibatch`小批量大小的一维向量的值，此标准期望一个范围在[0, C-1]的类别索引作为其目标。如果`ignore_index`指定了索引，此标准也接受这个类别索引，且这个索引可以超出类别的范围。


交叉熵损失
$$loss(x, class) = -log \left( \frac{\exp(x[class])}{\sum_j \exp(x[j])} \right) = -x[class]  + log \left( \sum_j \exp(x[j]) \right)$$


交叉熵损失（指定了`weight`的情况）
$$loss(x, class) = weight[class] \left( -x[class]  + log \left( \sum_j \exp(x[j]) \right) \right)$$

- 对于每个`minibatch`，损失是通过观察得到的平均值。如果指定了`wegiht`参数，那么这是一个加权平均值。
$$loss=\frac{\sum_{i=1}^N loss(i, class[i])} {\sum_{i=1}^N weight[class[i]]}$$

如果需要参考类别不均衡权重的更多信息，请参考Pytorch官方英文文档。

In [ ]:
import numpy as np

class CrossEntropyLoss():
    
    """
    初始化参数

    weight: (1, C) 设置每个类别不同的权重
    size_average: True or False 是否要对 loss 求平均，即除以batch_size
    """
    
    def __init__(self, weight=None, size_average=True):
        self.weight = weight
        self.size_average = size_average
        
    
    """
    计算交叉熵损失
        1. 计算目标分类结果发生的信息熵
        2. 计算所有可能发生事件的信息熵
        3. 取负对数似然作为损失值，损失越小，发生概率越大。
    
    补充说明
        这里input不需要经过softmax，直接对应于模型的输出

    input: (batch_size, C)，C是类别的总数，一个样本分成每一类的信息
    target: (batch_size, 1)，一个样本确定为某一类
    """
    
    def __call__(self, input, target):
        batch_loss = 0.
        for i in range(input.shape[0]):
            numerator = np.exp(input[i, target[i]])     # 分子
            denominator = np.sum(np.exp(input[i, :]))   # 分母
            loss = -np.log(numerator / denominator)     # 计算单个损失
            if self.weight:
                loss = self.weight[target[i]] * loss
            batch_loss += loss            # 损失累加
        if self.size_average == True:    # 整个 batch 的总损失是否要求平均
            batch_loss /= input.shape[0]
        return batch_loss

In [24]:
import torch 
sample = torch.tensor([[2.,3],[1,3]])
target = torch.tensor([0,1])

criterion = nn.CrossEntropyLoss(reduction='mean')
loss = criterion(sample, target)
print(loss)

tensor(0.7201)


#### 3.3 [nn.BCELoss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html?highlight=bceloss#torch.nn.BCELoss) 
二值交叉熵损失（Binary Cross Entropy）可用于自编码器中测量重构（reconstruction）的误差，这里目标值 $y_n$ 范围在[0, 1]。

$$l(x,y)=L={ \left[ l_1, l_2, ..., l_N \right] }^T$$

$$l_n = -w_n[y_n·\log x_n + (1-y_n)·\log (1-x_n)]$$

$$
l(x,y) =
    \begin{cases}
        mean(L), & \text{if reduction = 'mean'} \\
        sum(L), & \text{if reduction = 'sum'} \\
    \end{cases}
$$

    补充说明：`log`可能会出现无穷大的情况，则对应的梯度也是无穷大，在线性回归不能直接（straight-forward）使用。在数学上是未定义的（ mathematically undefined），Pytorch做出的处理将其对数输出钳位大于或等于-100。
- $x_n = 0$ 或 $x_n = 1$，则 $log(0) = -∞$，因为 $\lim_{x→0} = -∞$
- $y_n = 0$ 或 $1 - y_n = 0$，此时会出现0乘以无穷大。如果有一个无限大的损失值，那么我们在梯度中会有一个无限大的项，因为 $\lim_{x→0} \frac {d}{dx} \log (x) = ∞$

In [26]:
sample = torch.tensor([[2.0,3.0],[1,3]])
target = torch.tensor([0,1])

one_hot_target = torch.zeros(target.shape[0], max(target)+1)
one_hot_target[torch.arange(target.shape[0]), target] = 1
sigmoid_input = torch.sigmoid(sample)
print('sigmoid_input',sigmoid_input)
print('one_hot_target',one_hot_target)

criterion = nn.BCELoss()
loss = criterion(sigmoid_input, one_hot_target)
print('bce',loss)

sigmoid_input tensor([[0.8808, 0.9526],
        [0.7311, 0.9526]])
one_hot_target tensor([[1., 0.],
        [0., 1.]])
bce tensor(1.1343)


手动计算

In [28]:
import numpy as np

a = -(1*np.log(sigmoid_input[0][0]) + np.log(1-sigmoid_input[0][1]))/2
b = -(np.log(1-sigmoid_input[1][0]) + 1*np.log(sigmoid_input[1][1]))/2
(a+b)/ 2

tensor(1.1343)

#### 3.4 [nn.BCEWithLogitsLoss](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html?highlight=bcewithlogitsloss#torch.nn.BCEWithLogitsLoss)
逻辑二值交叉熵损失（Binary Cross Entropy With Logitis Loss），组合了`Sigmoid`和`BCELoss`。该版本比使用普通的Sigmoid和BCELoss的版本在数值上更稳定，因为通过将操作合并到一层中，我们利用了log-sum-exp技巧来实现数值稳定性。

$$l(x,y)=L={ \left[ l_1, l_2, ..., l_N \right] }^T$$

$$l_n = -w_n[y_n·\log \sigma(x_n)+ (1-y_n)·\log (1-\sigma(x_n))]$$

$$
l(x,y) =
    \begin{cases}
        mean(L), & \text{if reduction = 'mean'} \\
        sum(L), & \text{if reduction = 'sum'} \\
    \end{cases}
$$

##### 多标签分类
通过为positive的例子增加权重，可以在召回率（recall）和准确率（precision）之间进行权衡。在多标签分类（mluti-label classification）情况下，损失可描述为：

$$l_c(x,y)=L_c={ \left[ l_{1,c}, l_{2,c}, ..., l_{N,c} \right] }^T$$

$$l_{n,c} = -w_{n,c}[p_c y_{n,c}·\log \sigma(x_{n,c}) + (1-y_{n,c})·\log (1- \sigma(x_{n,c}))]$$

- `c`是类别数量（$c>1$为多标签分类，$c=1$为单标签二值分类）
- `n`是样本的batch大小
- $p_c$ 是对于类别正确答案的权重（$p_c > 1$ 增加召回率，$p_c < 1$ 增加准确率）

##### 实例分析
一个数据集包含$100$个正样例和$300$个负样例，然后`pos_weight`的权重等于 $\frac{300}{100} = 3$。就像损失包含了$3×100 = 300$个正样例。

In [47]:
target = torch.ones([10, 64], dtype=torch.float32)  # 64 classes, batch size = 10
output = torch.full([10, 64], 1.5)  # A prediction (logit)
pos_weight = torch.ones([64])  # All weights are equal to 1
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
criterion(output, target)  # -log(sigmoid(1.5))

tensor(0.2014)

BCELoss + Sigmoid

In [48]:
sigmoid = nn.Sigmoid()

torch.manual_seed(0)
sample = torch.randn(3,2)

torch.manual_seed(3)
target = torch.empty(3,2).random_(2)  # target one-hot type,such as tensor([0., 1.]).

sigmoid_input = sigmoid(sample)
criterion = nn.BCELoss()
loss = criterion(sigmoid_input,target)
loss

tensor(0.9232)

BCEWithLogtisLoss

In [49]:
criterion = nn.BCEWithLogitsLoss()
loss = criterion(sample, target)
loss

tensor(0.9232)

#### [nn.MultiLabelSoftMarginLoss](https://pytorch.org/docs/stable/generated/torch.nn.MultiLabelSoftMarginLoss.html?highlight=multilabelsoftmarginloss#torch.nn.MultiLabelSoftMarginLoss)
基于最大熵（max-entropy）的多个标签一对所有的损失。`input`和`target`的类型都为(N, C)。对于在minibatch的每个样本：

$$loss(x,y)= - \frac1C∗\sum_i y[i] ∗ \log((1+\exp(-x[i]))^{-1})+(1-y[i])∗\log(\frac{\exp(-x[i])}{1+\exp(-x[i])})$$
$\text{where}, i \in \{0, 1, ..., x.nElement() - 1 \}, y[i] \in \{ 0, 1 \}$

##### 多标签软边缘损失
支持一个样本含有多个标签，比如 target = [1,0,1]，代表该样本属于类别0和类别2

import torch 
from torch import nn

torch.manual_seed(0)
x = torch.randn(10, 3)
y = torch.FloatTensor(10, 3).random_(2)

bce_criterion = nn.BCEWithLogitsLoss(weight=None, reduce=False)
multi_criterion = nn.MultiLabelSoftMarginLoss(weight=None, reduce=False)

bce_loss = bce_criterion(x, y)
multi_loss = multi_criterion(x, y)

print('weight=None, bce_loss:\n',torch.mean(bce_loss, dim = 1))
print('weight=None, multi_loss:\n', multi_loss)

In [54]:
import torch 
from torch import nn

torch.manual_seed(0)
x = torch.randn(10, 3)
y = torch.FloatTensor(10, 3).random_(2)

bce_criterion = nn.BCEWithLogitsLoss(weight=None, reduction='none')
multi_criterion = nn.MultiLabelSoftMarginLoss(weight=None, reduction='none')

bce_loss = bce_criterion(x, y)
multi_loss = multi_criterion(x, y)

bce_loss, torch.mean(bce_loss, dim = 1), multi_loss

(tensor([[1.4068, 1.4269, 0.5757],
         [0.9334, 1.2050, 0.4058],
         [0.8636, 2.2291, 0.5449],
         [0.2490, 0.5334, 0.8590],
         [0.7549, 0.2547, 0.7675],
         [0.6389, 1.0463, 0.7091],
         [0.9695, 0.8250, 0.9370],
         [0.7509, 0.3585, 0.0941],
         [0.7456, 0.3734, 0.5588],
         [0.7197, 0.9884, 0.0953]]),
 tensor([1.1364, 0.8481, 1.2125, 0.5471, 0.5923, 0.7981, 0.9105, 0.4012, 0.5593,
         0.6012]),
 tensor([1.1364, 0.8481, 1.2125, 0.5471, 0.5923, 0.7981, 0.9105, 0.4012, 0.5593,
         0.6012]))

添加类的权重或者每个样本增加权重

In [55]:
#  the loss for class 1
class_weight = torch.FloatTensor([1.0, 2.0, 1.0])
#  the loss for last sample
element_weight = torch.FloatTensor([1.0]*9 + [2.0]).view(-1, 1)
element_weight = element_weight.repeat(1, 3)

bce_criterion_class = nn.BCEWithLogitsLoss(weight=class_weight, reduce=False)
multi_criterion_class = nn.MultiLabelSoftMarginLoss(weight=class_weight, reduce=False)
bce_criterion_element = nn.BCEWithLogitsLoss(weight=element_weight, reduce=False)
multi_criterion_element = nn.MultiLabelSoftMarginLoss(weight=element_weight, reduce=False)

bce_loss_class = bce_criterion_class(x, y)
multi_loss_class = multi_criterion_class(x, y)
bce_loss_element = bce_criterion_element(x, y)
multi_loss_element = multi_criterion_element(x, y)

print("class weight, BCE loss:\n", torch.mean(bce_loss_class,dim=1))
print("class weight, multi loss:\n",multi_loss_class)
print("element weight, BCE loss:\n", torch.mean(bce_loss_element,dim=1))
print("element weight, multi loss:\n",multi_loss_element)

class weight, BCE loss:
 tensor([1.6121, 1.2497, 1.9556, 0.7249, 0.6772, 1.1468, 1.1855, 0.5207, 0.6838,
        0.9306])
class weight, multi loss:
 tensor([1.6121, 1.2497, 1.9556, 0.7249, 0.6772, 1.1468, 1.1855, 0.5207, 0.6838,
        0.9306])
element weight, BCE loss:
 tensor([1.1364, 0.8481, 1.2125, 0.5471, 0.5923, 0.7981, 0.9105, 0.4012, 0.5593,
        1.2023])
element weight, multi loss:
 tensor([1.1364, 0.8481, 1.2125, 0.5471, 0.5923, 0.7981, 0.9105, 0.4012, 0.5593,
        1.2023])
